In [1]:
!ls

Analysis.ipynb        test.csv
sample_submission.csv train.csv


In [2]:
import pandas as pd

df = pd.read_csv('train.csv')
df.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [3]:
df = df.drop(columns=['id'], errors='ignore')
df.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [127]:
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin


nom_cols = ['nom_0', 'nom_1', 'nom_2', 'nom_3']
bin_cols = [x for x in df.columns if 'bin' in x]
ord_cols = [x for x in df.columns if 'ord' in x]



# Create a custom Mixin 
def transform_df(X):
    X['bin_3'] = X['bin_3'].map(lambda x : 1 if x=='T' else 0)
    X['bin_4'] = X['bin_4'].map(lambda x : 1 if x=='Y' else 0)    
    
    # Dummy Encode the Categorical Values
    X_dummies = pd.get_dummies(X[nom_cols], drop_first=True)

    # Create the transformed DF
    res = X[bin_cols]
    res = pd.concat([res, X_dummies], axis=1)
    res['target'] = X['target']
    
    
    return res

    

a= transform_df(df)
a.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0_Green,nom_0_Red,nom_1_Polygon,nom_1_Square,nom_1_Star,...,nom_2_Dog,nom_2_Hamster,nom_2_Lion,nom_2_Snake,nom_3_China,nom_3_Costa Rica,nom_3_Finland,nom_3_India,nom_3_Russia,target
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
1,0,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
3,0,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [128]:
# Create Train-Test Split
from sklearn.model_selection import train_test_split

X = a.drop(columns='target')
y = a['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [129]:
# Train Model

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train, y_train)

In [150]:
# Baseline. Our data has an imbalance towards 0's.
# If we guess 0 always, we end up with accuracy ~70%
base_acc = y_train.sum()/len(y_train)
print("Baseline model accuracy : {}".format(1-base_acc))

# Evaluate Model
y_pred = lr.predict(X_test)

# Accuracy of our model
acc = (y_pred == y_test).sum()/len(y_test)
print("The resulting Accuracy is : {}".format(acc))

Baseline model accuracy : 0.6936069651741293
The resulting Accuracy is : 0.6958585858585858
